# ニューラルネットワークによるMNISTデータセットの認識実験

---
## 目的
多層パーセプトロン (Multi Layer Perceptoron; MLP) を用いてMNISTデータセットに対する文字認識を行う．

## 対応するチャプター
* 6序文: 全結合型ニューラルネットワーク
* 6.2.2: マルチヌーイ分布出力のためのソフトマックスユニット
* 6.3 (6.3.1): ReLUとその一般化
* 6.3 (6.3.2): ロジスティックシグモイドとハイパボリックタンジェント（tanh）
* 10.9 (10.9.2): 複数時間スケールのためのLeakyユニットとその他の手法（Leaky ReLU）
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gzip

## データセットのダウンロードと読み込み

今回の実験では，MNISTデータセットを使用します．

[MNIST dataset](http://yann.lecun.com/exdb/mnist/)

まずはじめに，`wget`コマンドを使用して，MNISTデータセットをダウンロードします．

In [ ]:
!wget https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz -O train-images-idx3-ubyte.gz
!wget https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz -O train-labels-idx1-ubyte.gz
!wget https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz -O t10k-images-idx3-ubyte.gz
!wget https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz -O t10k-labels-idx1-ubyte.gz

次に，ダウンロードしたファイルからデータを読み込みます．
読み込み方は，公式のHPに公開されているように，`gzip`でファイルを展開し，`frombuffer`関数でnumpy arrayとして読み込みます．

In [ ]:
# load images
with gzip.open('train-images-idx3-ubyte.gz', 'rb') as f:
    x_train = np.frombuffer(f.read(), np.uint8, offset=16)
x_train = x_train.reshape(-1, 784)

with gzip.open('t10k-images-idx3-ubyte.gz', 'rb') as f:
    x_test = np.frombuffer(f.read(), np.uint8, offset=16)
x_test = x_test.reshape(-1, 784)

with gzip.open('train-labels-idx1-ubyte.gz', 'rb') as f:
    y_train = np.frombuffer(f.read(), np.uint8, offset=8)

with gzip.open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
    y_test = np.frombuffer(f.read(), np.uint8, offset=8)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

### MNISTデータセットの表示
MNISTデータセットに含まれる画像を表示してみます．
ここでは，matplotlibを用いて複数の画像を表示させるプログラムを利用します．

In [ ]:
plt.clf()
fig = plt.figure(figsize=(14, 1.4))
for c in range(10):
    ax = fig.add_subplot(1, 10, c + 1)
    ax.imshow(x_train[c].reshape(28, 28), cmap=plt.get_cmap('gray'))
    ax.set_axis_off()
plt.show()

## ネットワークモデルの定義
次に，ニューラルネットワーク（多層パーセプトロン）を定義します．

まずはじめに，ネットワークの定義に必要な関数を定義します．

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    grad = np.zeros(x.shape)
    grad[x >= 0] = 1
    return grad

def leaky_relu(x, negative_slope=0.01):
    return np.maximum(0, x) + 0.01 * np.minimum(0, x)
    
def leaky_relu_grad(x, negative_slope=0.01):
    grad = np.zeros(x.shape)
    grad[x >= 0] = 1
    grad[x < 0] = negative_slope
    return grad

def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    
def tanh_grad(x):
    return 1 - x**2

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def softmax_cross_entropy(x, t):
    y = softmax(x)
    return cross_entropy(y, t)

def multiclass_classification_accuracy(pred, true):
    clf_res = np.argmax(pred, axis=1)
    return np.sum(clf_res == true).astype(np.float32)

次に，上で定義した関数を用いてネットワークを定義します．
ここでは，入力層，中間層，出力層から構成される多層パーセプトロンとします．

入力層と中間層，出力層のユニット数は引数として与え，それぞれ`input_size`，`hidden_size`, `output_size`とします．
また，ネットワーク内で使用する活性化関数を選択する`act_func`を`['relu', 'lrelu', 'tanh']`のうちから一つ選択します．
活性化関数に`lrelu`（Leaky ReLU）を選択した場合の負の値に対する漏れ値（`negative_slope`）を設定します．
そして，`__init__`関数を用いて，ネットワークのパラメータを初期化します．
`w1`および`w2`は各層の重みで，`b1`および`b2`はバイアスを表しています．
重みは`randn`関数で，標準正規分布に従った乱数で生成した値を保有する配列を生成します．
バイアスは`zeros`関数を用いて，要素が全て0の配列を生成します．

そして，`forward`関数で，データを入力して結果を出力するための演算を定義します．

次に，`backward`関数ではパラメータの更新量を計算します．
まず，ネットワークの出力結果と教師ラベルから，誤差`dy`を算出します．
この時，教師ラベルをone-hotベクトルへ変換し，各ユニットの出力との差を取ることで，`dy`を計算しています．
その後，連鎖律に基づいて，出力層から順番に勾配を計算していきます．
このとき，パラメータの更新量を`self.grads`へ保存しておきます．

最後に`update_parameters`関数で，更新量をもとにパラメータの更新を行います．

In [ ]:
class MLPMultinoulli:
    
    def __init__(self, input_size, hidden_size, output_size, act_func='relu', negative_slope=0.01, w_std=0.01):
        
        assert act_func in ['relu', 'lrelu', 'tanh'], "act_func should be selected from ['relu', 'lrelu', 'tanh']"
        self.act_func = act_func
        self.negative_slope = negative_slope
        
        self.w1 = w_std * np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)
        self.w2 = w_std * np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros(output_size)
        
        if self.act_func == 'relu':
            self.act = relu
            self.act_grad = relu_grad
        elif self.act_func == 'lrelu':
            self.act = leaky_relu
            self.act_grad = leaky_relu_grad
        elif self.act_func == 'tanh':
            self.act = tanh
            self.act_grad = tanh_grad

        self.grads = {}

    def forward(self, x):
        self.h1 = np.dot(x, self.w1) + self.b1
        if self.act_func == 'lrelu':
            self.h2 = self.act(self.h1, self.negative_slope)
        else:
            self.h2 = self.act(self.h1)
        self.h3 = np.dot(self.h2, self.w2) + self.b2
        return self.h3

    def backward(self, x, t):
        batch_size = x.shape[0]
        
        # forward
        _ = self.forward(x)
        y = softmax(self.h3)
        
        # backward
        self.grads = {}
        
        t = np.identity(10)[t]
        
        dy = (y - t) / batch_size
        
        d_h2 = np.dot(dy, self.w2.T)
        self.grads['w2'] = np.dot(self.h2.T, dy)
        self.grads['b2'] = np.sum(dy, axis=0)
        
        if self.act_func == 'lrelu':
            d_h1 = self.act_grad(self.h2, self.negative_slope) * d_h2
        else:
            d_h1 = self.act_grad(self.h2) * d_h2
        
        self.grads['w1'] = np.dot(x.T, d_h1)
        self.grads['b1'] = np.sum(d_h1, axis=0)
        
    def update_parameters(self, lr=0.1):
        self.w1 -= lr * self.grads['w1']
        self.b1 -= lr * self.grads['b1']
        self.w2 -= lr * self.grads['w2']
        self.b2 -= lr * self.grads['b2']

## ネットワークの作成と学習の準備
上のプログラムで定義したネットワークを作成します．


まず，中間層と出力層のユニット数を定義します．
ここでは，入力層のユニット数`input_size`を学習データの次元，中間層のユニット数`hidden_size`を128，出力層のユニット数`output_size`を10とします．
活性化関数はReLUを使用します．

各層のユニット数を`MLPMultinoulli`クラスの引数として与え，ネットワークを作成します．

In [ ]:
input_size = x_train.shape[1]
hidden_size = 128
output_size = 10
model = MLPMultinoulli(input_size=input_size, hidden_size=hidden_size, output_size=output_size, act_func='tanh')

## 学習
読み込んだMNISTデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を100，学習エポック数を20とします．

学習データは毎回ランダムに決定するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x_batch`と`y_batch`とします．
学習モデルに`x_batch`を与えて，`h`を取得します．
取得した`h`は精度および誤差を算出するための関数へと入力され，値を保存します．
そして，誤差を`backward`関数で逆伝播し，`update_parameters`でネットワークの更新を行います．

In [ ]:
num_train_data = x_train.shape[0]
batch_size = 100
epoch_num = 20

for epoch in range(1, epoch_num + 1):
    
    sum_accuracy = 0.0
    sum_loss = 0.0
    
    perm = np.random.permutation(num_train_data)
    for i in range(0, num_train_data, batch_size):
        x_batch = x_train[perm[i:i+batch_size]]
        y_batch = y_train[perm[i:i+batch_size]]
        
        h = model.forward(x_batch)
        sum_accuracy += multiclass_classification_accuracy(h, y_batch)
        sum_loss += softmax_cross_entropy(h, y_batch)
        
        model.backward(x_batch, y_batch)
        model.update_parameters(lr=0.001)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch,
                                                               sum_loss / num_train_data,
                                                               sum_accuracy / num_train_data))

## テスト
学習したネットワークを用いて，テストデータに対する認識率の確認を行います．

In [ ]:
count = 0
num_test_data = x_test.shape[0]

for i in range(num_test_data):
    x = np.array([x_test[i]], dtype=np.float32)
    t = y_test[i]
    y = model.forward(x)
    
    pred = np.argmax(y.flatten())
    
    if pred == t:
        count += 1
        
print("test accuracy: {}".format(count / num_test_data))

## 課題
1. ネットワークの中間層のユニット数を変更して認識性能の変化を確認しましょう
2. 使用する活性化関数を変更して認識性能の変化を確認しましょう